In [3]:
# Dependencies
import requests
import json
import pandas as pd
import numpy as np

from pprint import pprint

# Google developer API key
from config import gkey

# Read in Arlington County Housing Assessment data
arl_data = pd.read_csv("2018_Arl_Asmt.csv")
arl_data.head()

,IvwPropertyAssessmentHistoryKey,ProvalLrsnId,RealEstatePropertyCode,MasterRealEstatePropertyCode,ReasPropertyStatusCode,PropertyClassTypeCode,PropertyClassTypeDsc,PropertyStreetNbrNameText,PropertyStreetNbr,PropertyStreetNbrSuffixCode,...,PropertyExpiredInd,CommercialInd,DistrictNbr,TaxExemptionTypeDsc,CondominiumProjectName,TaxBalanceAmt,AssessedYearDate,TotalAssessedAmt,AssessmentDate,AssessmentChangeReasonTypeDsc
0,36,133,1001006,1001006,T,510,510-Res - Vacant(SF & Twnhse),N ROCHESTER ST,NaN,NaN,...,False,False,0G,NaN,NaN,0.0,2018,16600,1/1/2018 0:00,01- Annual
1,73,134,1001007,1001007,A,511,511-Single Family Detached,3007 N ROCHESTER ST,3007.0,NaN,...,False,False,0G,NaN,NaN,0.0,2018,1772500,1/1/2018 0:00,01- Annual
2,110,136,1001009,1001009,A,511,511-Single Family Detached,6547 WILLIAMSBURG BLVD,6547.0,NaN,...,False,False,0G,NaN,NaN,0.0,2018,773000,1/1/2018 0:00,01- Annual
3,147,137,1001010,1001010,A,511,511-Single Family Detached,6541 WILLIAMSBURG BLVD,6541.0,NaN,...,False,False,0G,NaN,NaN,0.0,2018,991300,1/1/2018 0:00,01- Annual
4,184,139,1001012,1001012,A,511,511-Single Family Detached,3518 N SOMERSET ST,3518.0,NaN,...,False,False,0G,NaN,NaN,0.0,2018,1341700,1/1/2018 0:00,01- Annual


In [4]:
metro_list = pd.DataFrame(
   {'metro': ['Arlington Cemetery','Ballston MU','Clarendon','Court House','Crystal City','East Falls Church','Pentagon','Pentagon City','Ronald Reagan Washington National Airport','Rosslyn','Virginia Square GMU'],
    'street': ['1000 North Memorial Drive', '4230 Fairfax Drive', '3100 Wilson Blvd', '2100 Wilson Blvd', '1750 South Clark St.', '2001 N. Sycamore St.', '2 South Rotary Road', '1250 South Hayes St.', '2400 S. Smith Blvd.', '1850 N. Moore Street', '3600 Fairfax Drive'],
    'city': ['Arlington', 'Arlington', 'Arlington', 'Arlington', 'Arlington', 'Arlington', 'Arlington', 'Arlington', 'Arlington', 'Arlington', 'Arlington'],
    'state': ['VA', 'VA', 'VA', 'VA', 'VA', 'VA', 'VA', 'VA', 'VA', 'VA', 'VA'],
    'zip': ['22211', '22201', '22201', '22201', '22202', '22205', '22202', '22202', '22202', '22209', '22201']
   })

metro_list["Lat"] = ""
metro_list["Lng"] = ""

metro_list

,metro,street,city,state,zip,Lat,Long
0,Arlington Cemetery,1000 North Memorial Drive,Arlington,VA,22211,,
1,Ballston MU,4230 Fairfax Drive,Arlington,VA,22201,,
2,Clarendon,3100 Wilson Blvd,Arlington,VA,22201,,
3,Court House,2100 Wilson Blvd,Arlington,VA,22201,,
4,Crystal City,1750 South Clark St.,Arlington,VA,22202,,
5,East Falls Church,2001 N. Sycamore St.,Arlington,VA,22205,,
6,Pentagon,2 South Rotary Road,Arlington,VA,22202,,
7,Pentagon City,1250 South Hayes St.,Arlington,VA,22202,,
8,Ronald Reagan Washington National Airport,2400 S. Smith Blvd.,Arlington,VA,22202,,
9,Rosslyn,1850 N. Moore Street,Arlington,VA,22209,,


In [6]:
# create a params dict that will be updated with new city each iteration
params = {"key": gkey}

# Loop through the cities_pd and run a lat/long search for each city
for index, row in metro_list.iterrows():
    base_url = "https://maps.googleapis.com/maps/api/geocode/json"

    street = row['street']
    city = row['city']
    state = row['state']
    zip = row['zip']

    # update address key value
    params['address'] = f"{street},{city},{state},{zip}"

    # make request
    metros_lat_lng = requests.get(base_url, params=params)
    
    # print the cities_lat_lng url, avoid doing for public github repos in order to avoid exposing key
    # print(cities_lat_lng.url)
    
    # convert to json
    metros_lat_lng = metros_lat_lng.json()

    metro_list.loc[index, "Lat"] = metros_lat_lng["results"][0]["geometry"]["location"]["lat"]
    metro_list.loc[index, "Lng"] = metros_lat_lng["results"][0]["geometry"]["location"]["lng"]

# Visualize to confirm lat lng appear
metro_list.head(15)

,metro,street,city,state,zip,Lat,Long,Lng
0,Arlington Cemetery,1000 North Memorial Drive,Arlington,VA,22211,38.8846,,-77.063173
1,Ballston MU,4230 Fairfax Drive,Arlington,VA,22201,38.882,,-77.111477
2,Clarendon,3100 Wilson Blvd,Arlington,VA,22201,38.8871,,-77.095230
3,Court House,2100 Wilson Blvd,Arlington,VA,22201,38.8916,,-77.084877
4,Crystal City,1750 South Clark St.,Arlington,VA,22202,38.8572,,-77.051888
5,East Falls Church,2001 N. Sycamore St.,Arlington,VA,22205,38.8855,,-77.156338
6,Pentagon,2 South Rotary Road,Arlington,VA,22202,38.871,,-77.055949
7,Pentagon City,1250 South Hayes St.,Arlington,VA,22202,38.8631,,-77.060109
8,Ronald Reagan Washington National Airport,2400 S. Smith Blvd.,Arlington,VA,22202,38.8497,,-77.043455
9,Rosslyn,1850 N. Moore Street,Arlington,VA,22209,38.8969,,-77.071978
